In [75]:
import pybaseball 

from pybaseball.lahman import batting, pitching, people, fielding

import pandas as pd
pd.set_option('display.max_columns', 500)

### import and prep pitch data

In [96]:
pitch_data = pd.read_csv("../data/pitch_data_2019.csv")

# games in Japan had no player position data
pitch_data = pitch_data[pitch_data['fielder_2.1'] == pitch_data['fielder_2.1']]

# make sure the pitch sequence is in order
pitch_data.sort_values(by=['game_date', 'game_pk', 'inning', 'at_bat_number', 'pitch_number', 'outs_when_up'], ascending=True, inplace=True)

# keep only pitches that end in an "event" (hit, out, etc.)
pitch_data = pitch_data[pitch_data['events'] == pitch_data['events']]

cols_to_keep = ['game_date', 'game_pk', 'inning', 'inning_topbot', 'on_1b', 'on_2b', 'on_3b', 'outs_when_up', 'events', 'fld_score', 'bat_score', 
                'hc_x', 'hc_y', 'hit_location', 'batter',
                'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9']

pitch_data = pitch_data[cols_to_keep]

pitch_data.rename(columns={'on_1b': 'playerId_on_1b', 'on_2b': 'playerId_on_2b', 'on_3b': 'playerId_on_3b', 'hit_location': 'first_fielder'}, inplace=True)

for col in ['playerId_on_1b', 'playerId_on_2b', 'playerId_on_3b']:
    pitch_data[col[9:]] = pitch_data[col] == pitch_data[col]

pitch_data['fld_bat_score_diff'] = pitch_data['fld_score'] - pitch_data['bat_score']

pitch_data = pitch_data[['game_date', 'game_pk', 'inning', 'inning_topbot', 'on_1b', 'on_2b', 'on_3b', 'outs_when_up', 'fld_bat_score_diff',
                         'events', 'hc_x', 'hc_y', 'first_fielder', 
                         'batter', 'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 
                         'playerId_on_1b', 'playerId_on_2b', 'playerId_on_3b']]

for col in ['first_fielder', 'playerId_on_1b', 'playerId_on_2b', 'playerId_on_3b']:
    pitch_data[col] = pitch_data[col].fillna(value=0.)

for col in ['inning', 'outs_when_up', 'batter', 'first_fielder',
            'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 
            'playerId_on_1b', 'playerId_on_2b', 'playerId_on_3b']:
    pitch_data[col] = pitch_data[col].astype(int)

print(pitch_data.shape)
pitch_data.head()

(185334, 26)


,game_date,game_pk,inning,inning_topbot,on_1b,on_2b,on_3b,outs_when_up,fld_bat_score_diff,events,hc_x,hc_y,first_fielder,batter,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,playerId_on_1b,playerId_on_2b,playerId_on_3b
731783,2019-03-28,565220.0,1,Top,False,False,False,0,0.0,field_out,152.35,165.76,3,624428,622491,571466,458015,606299,553993,578428,608385,594988,624577,0,0,0
731779,2019-03-28,565220.0,1,Top,False,False,False,1,0.0,strikeout,NaN,NaN,2,466320,622491,571466,458015,606299,553993,578428,608385,594988,624577,0,0,0
731774,2019-03-28,565220.0,1,Top,False,False,False,2,0.0,walk,NaN,NaN,0,572816,622491,571466,458015,606299,553993,578428,608385,594988,624577,0,0,0
731771,2019-03-28,565220.0,1,Top,True,False,False,2,0.0,strikeout,NaN,NaN,2,605137,622491,571466,458015,606299,553993,578428,608385,594988,624577,572816,0,0
731770,2019-03-28,565220.0,1,Bot,False,False,False,0,0.0,field_out,74.84,96.79,7,608385,592791,465041,605137,624428,628356,570481,572816,543776,466320,0,0,0


### import batters

In [45]:
batters_df = batting()
batters_df = batters_df[batters_df['yearID'] == 2012]

people_df = people()
people_df = people_df[['playerID', 'nameLast', 'nameGiven']]

position_df = fielding()
position_df = position_df[['playerID', 'POS']]

people_df = pd.merge(people_df, position_df, how='inner', on='playerID')
people_df.drop_duplicates(inplace=True)

batters_df = pd.merge(people_df, batters_df, how='inner', on='playerID')

batters_df = batters_df.drop(['yearID', 'stint', 'teamID', 'lgID'], axis=1)

# filter out pitchers and then drop duplicates (players that played multiple positions)
batters_df = batters_df[batters_df['POS'] != 'P']
batters_df.drop('POS', axis=1, inplace=True)
batters_df.drop_duplicates(inplace=True)

batters_df.sort_values(by='AB', inplace=True, ascending=False)
batters_df.reset_index(inplace=True, drop=True)

# make a list of "Major League Players" (i.e., 13 * 30 = 390 position players with most ABs)
mlb_players_list = batters_df['playerID'].iloc[:390].tolist()
print(f"Number of Major League Position Players: {len(mlb_players_list)}")

# make a list of "replacement-level players"
replacement_players_list = batters_df['playerID'].iloc[390:].tolist()
print(f"Number of Replacement Position Players: {len(replacement_players_list)}")

print(batters_df.shape)
batters_df.head()

Number of Major League Position Players: 390
Number of Replacement Position Players: 321
(711, 20)


,playerID,nameLast,nameGiven,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,jeterde01,Jeter,Derek Sanderson,159,683,99,216,32,0,15,58.0,9.0,4.0,45,90.0,1.0,5.0,6.0,1.0,24.0
1,hardyjj01,Hardy,James Jerry,158,663,85,158,30,2,22,68.0,0.0,0.0,38,106.0,4.0,3.0,7.0,2.0,21.0
2,kinslia01,Kinsler,Ian Michael,157,655,105,168,42,5,19,72.0,21.0,9.0,60,90.0,0.0,10.0,1.0,5.0,14.0
3,jonesad01,Jones,Adam LaMarque,162,648,103,186,39,3,32,82.0,16.0,7.0,34,126.0,0.0,13.0,0.0,2.0,15.0
4,castrst01,Castro,Starlin DeJesus,162,646,78,183,29,12,14,78.0,25.0,13.0,36,100.0,5.0,4.0,0.0,5.0,15.0


### import pitchers

In [53]:
pitchers_df = pitching()
pitchers_df = pitchers_df[pitchers_df['yearID'] == 2012]

pitchers_df = pd.merge(people_df, pitchers_df, how='inner', on='playerID')

pitchers_df = pitchers_df.drop(['yearID', 'stint', 'teamID', 'lgID'], axis=1)

# filter out pitchers and then drop duplicates (players that played multiple positions)
pitchers_df = pitchers_df[pitchers_df['POS'] == 'P']
pitchers_df.drop('POS', axis=1, inplace=True)
pitchers_df.drop_duplicates(inplace=True)

pitchers_df.sort_values(by='BFP', inplace=True, ascending=False)
pitchers_df.reset_index(inplace=True, drop=True)

# make a list of "Major League Pitchers" (i.e., 12 * 30 = 360 position players with most ABs)
mlb_pitchers_list = pitchers_df['playerID'].iloc[:360].tolist()
print(f"Number of Major League Pitchers: {len(mlb_pitchers_list)}")

# make a list of "replacement-level players"
replacement_pitchers_list = pitchers_df['playerID'].iloc[360:].tolist()
print(f"Number of Replacement Pitchers: {len(replacement_pitchers_list)}")

print(pitchers_df.shape)
pitchers_df.head()

Number of Major League Pitchers: 360
Number of Replacement Pitchers: 362
(722, 28)


,playerID,nameLast,nameGiven,W,L,G,GS,CG,SHO,SV,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
0,verlaju01,Verlander,Justin Brooks,17,8,33,33,6,1,0,...,2.0,2,5.0,1,956.0,0,81,4.0,3.0,16.0
1,shielja02,Shields,James Anthony,15,10,33,33,3,2,0,...,2.0,7,11.0,1,944.0,0,103,3.0,2.0,20.0
2,hernafe02,Hernandez,Felix Abraham,13,9,33,33,5,5,0,...,0.0,13,12.0,2,939.0,0,84,2.0,2.0,23.0
3,dickera01,Dickey,Robert Allen,20,6,34,33,5,3,0,...,2.0,4,9.0,1,927.0,1,78,9.0,7.0,25.0
4,richacl01,Richard,Clayton Colby,14,14,33,33,1,1,0,...,4.0,4,6.0,2,910.0,0,110,3.0,6.0,22.0


In [56]:
test = people()

test.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01
